In [ ]:
"""
Resampling Sentinel Data and Clip
"""

import os

d = {
    '/mnt/c/gwork/osm2lulc_imgcls/sentinel/c_S2B_MSIL2A_20190629T112119' : {
        "FORMAT" : '.jp2',
        'FOLDER' : '/mnt/c/gwork/osm2lulc_imgcls/sentinel/o_S2B_MSIL2A_20190629T112119',
        'REF'    : 'T29SNC_20190629T112119_B02_10m.jp2',
        'CLIP'   : '/mnt/c/gwork/osm2lulc_imgcls/lmt/ext_lisboa_wgs.shp'
    },
}

from glass.gt.nop.rmp import match_cellsize_and_clip
from glass.pyt.oss import lst_ff

for k in d:
    bands = lst_ff(d[k]["FOLDER"], file_format=d[k]["FORMAT"])
    match_cellsize_and_clip(
        bands, os.path.join(d[k]['FOLDER'], d[k]['REF']),
        k#, clipShp=d[k]['CLIP']
    )

In [ ]:
"""
Mosaic Sentinel-2
"""

import os
from zipfile import ZipFile
from glass.pyt.oss import lst_ff, mkdir, fprop, copy_file
from glass.gt.nop.rmp import match_cellsize_and_clip
from glass.gt.nop.mos import bnds_to_mosaic

intbands = [
    'AOT_10m', 'B02_10m', 'B03_10m', 'B04_10m', 'B08_10m', 'WVP_10m',
    'B05_20m', 'B06_20m', 'B07_20m', 'B8A_20m', 'B11_20m', 'B12_20m',
    'SCL_20m', 'WVP_20m',
    'B09_60m', 'B01_60m'
]

mos_folder = '/mnt/c/gwork/osm2lulc_imgcls/sentinel'

clip_shp = '/mnt/c/gwork/osm2lulc_imgcls/lmt/lmt_lisboa.shp'

out_folder = '/mnt/c/gwork/osm2lulc_imgcls/sentinel'

ref_raster = '/mnt/c/gwork/osm2lulc_imgcls/lmt/rst_lisboa.tif'

# List all images in folder
imgs = lst_ff(mos_folder, file_format='.zip')

# Extract all bands from all images
img_bands = {}
for i in imgs:
    # Create folder
    i_key = fprop(i, 'fn')[:43]
    fld = mkdir(os.path.join(mos_folder, i_key))
    
    img_bands[i_key] = {}
    
    with ZipFile(i, 'r') as zip_o:
        zipff = zip_o.namelist()
        
        rbands = []
        for b in intbands:
            for f in zipff:
                if b in f:
                    ob = os.path.join(fld, os.path.basename(f))
                    zip_o.extract(f, path=fld, pwd=None)
                    copy_file(
                        os.path.join(fld, f), ob
                    )
                    
                    if "BANDS" not in img_bands[i_key]:
                        img_bands[i_key]["BANDS"] = [ob]
                    else:
                        img_bands[i_key]["BANDS"].append(ob)
                    
                    if b == 'B02_10m':
                        img_bands[i_key]["REF"] = ob
                    
                    break

In [ ]:
# Resample and clip Bands

for i in img_bands:
    img_bands[i]["BANDS"] = match_cellsize_and_clip(
        img_bands[i]["BANDS"], img_bands[i]["REF"],
        os.path.join(mos_folder, i),
        clipShp=clip_shp
    )

# Mosaic Bands

mos_bands = {}

for b in intbands:
    mos_bands[b] = []
    for i in img_bands:
        for _b in img_bands[i]["BANDS"]:
            if b in _b:
                mos_bands[b].append(_b)
                break

bnds_to_mosaic(mos_bands, out_folder, ref_raster)